# Question 1: Which states are most similar in terms of their national park type distribution?

We can create vectors for each state based on the type and quantity of national parks they have (National Historic Monument, National Lakeshore, etc). We can create a network of the states by calculating pairwise similarity of the vectors, where each node is a state/region and the edges are weighted by similarity. Then we complete a network analysis of the state nodes to find the state with highest between-
ness, closeness centrality, etc.

In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.tsa.seasonal import seasonal_decompose, DecomposeResult
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import seaborn as sns
# suppress all warnings
import warnings
warnings.filterwarnings("ignore")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


To start, let's read in the time series data.

In [4]:
pd.read_csv('timeseries_data.csv',index_col='Unnamed: 0')

,ParkName,UnitCode,ParkType,Region,state_2,visit_type,2003-01-31 00:00:00,2003-02-28 00:00:00,2003-03-31 00:00:00,2003-04-30 00:00:00,...,2023-03-31 00:00:00,2023-04-30 00:00:00,2023-05-31 00:00:00,2023-06-30 00:00:00,2023-07-31 00:00:00,2023-08-31 00:00:00,2023-09-30 00:00:00,2023-10-31 00:00:00,2023-11-30 00:00:00,2023-12-31 00:00:00
0,Abraham Lincoln Birthplace NHP,ABLI,National Historical Park,Southeast,KY,Backcountry,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Abraham Lincoln Birthplace NHP,ABLI,National Historical Park,Southeast,KY,ConcessionerCamping,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Abraham Lincoln Birthplace NHP,ABLI,National Historical Park,Southeast,KY,ConcessionerLodging,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Abraham Lincoln Birthplace NHP,ABLI,National Historical Park,Southeast,KY,MiscellaneousOvernightStays,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Abraham Lincoln Birthplace NHP,ABLI,National Historical Park,Southeast,KY,NonRecreationHours,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4340,Zion NP,ZION,National Park,Intermountain,UT,NonRecreationVisits,1860.0,1680.0,1860.0,1800.0,...,1860.0,1800.0,1860.0,1800.0,1860.0,1860.0,1800.0,1860.0,1800.0,1860.0
4341,Zion NP,ZION,National Park,Intermountain,UT,RVCampers,653.0,833.0,3108.0,9396.0,...,7377.0,7932.0,7072.0,9835.0,9539.0,9225.0,8306.0,9191.0,7035.0,4389.0
4342,Zion NP,ZION,National Park,Intermountain,UT,RecreationHours,437574.0,431814.0,970378.0,1765989.0,...,2237847.0,2861784.0,3134999.0,3585357.0,3644234.0,3027979.0,3063140.0,3149423.0,1896028.0,1343422.0
4343,Zion NP,ZION,National Park,Intermountain,UT,RecreationVisits,71233.0,72781.0,143284.0,244910.0,...,336226.0,450819.0,514283.0,564036.0,561026.0,458449.0,491799.0,504584.0,284978.0,212443.0
